# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
from psegs.exp.fused_lidar import FusedWorldCloudTableBase

class SemanticKITTIFusedWorldCloudTable(FusedWorldCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        return task_rdd
        


## Start Spark

In [3]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-11 16:09:48,658	oarph 27442 : Using source root /opt/psegs/psegs 
2021-02-11 16:09:48,659	oarph 27442 : Using source root /opt/psegs 
2021-02-11 16:09:48,811	oarph 27442 : Generating egg to /tmp/tmpnhwy7s7y_oarphpy_eggbuild ...
2021-02-11 16:09:48,919	oarph 27442 : ... done.  Egg at /tmp/tmpnhwy7s7y_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [4]:
seg_uris = SemanticKITTIFusedWorldCloudTable.get_all_segment_uris()
sd_rdd = SemanticKITTIFusedWorldCloudTable._get_segment_datum_rdd_or_df(spark, seg_uris)
print(sd_rdd.count())

In [5]:
# SemanticKITTIFusedWorldCloudTable.build(spark=spark)

2021-02-11 16:09:52,680	ps   27442 : Found Sequence 00 with 4541 scans
2021-02-11 16:09:52,683	ps   27442 : Found Sequence 01 with 1101 scans
2021-02-11 16:09:52,685	ps   27442 : Found Sequence 02 with 4661 scans
2021-02-11 16:09:52,687	ps   27442 : Found Sequence 03 with 801 scans
2021-02-11 16:09:52,689	ps   27442 : Found Sequence 05 with 2761 scans
2021-02-11 16:09:52,690	ps   27442 : Found Sequence 06 with 1101 scans
2021-02-11 16:09:52,692	ps   27442 : Found Sequence 07 with 1101 scans
2021-02-11 16:09:52,694	ps   27442 : Found Sequence 09 with 1591 scans
2021-02-11 16:09:52,695	ps   27442 : Found Sequence 10 with 1201 scans
2021-02-11 16:09:52,696	ps   27442 : Found 18859 total scans
2021-02-11 16:09:52,696	ps   27442 : SemanticKITTIFusedWorldCloudTable building fused world clouds ...
2021-02-11 16:09:52,696	ps   27442 : ... have 9 segments to fuse ...
2021-02-11 16:09:52,697	ps   27442 : ... working on 00 ...
2021-02-11 16:09:52,698	ps   27442 : ... have fused cloud; skipping! /

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 0.0 failed 1 times, most recent failure: Lost task 8.0 in stage 0.0 (TID 8, 192.168.65.3, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/pyspark-3.0.1-py3.8.egg/pyspark/sql/session.py", line 612, in prepare
  File "/usr/local/lib/python3.